In [ ]:
%matplotlib inline
import shapely
from matplotlib import pyplot as plt
import matplotlib as mpl
from matplotlib.patches import ConnectionPatch
from mpl_toolkits.mplot3d import Axes3D
from shapely.geometry import LinearRing, LineString, Polygon, Point, MultiLineString, MultiPoint
from shapely.ops import linemerge, polygonize, transform
from shapely import affinity
from IPython.display import display, Markdown, Latex, Math, display_markdown
from descartes import PolygonPatch
import numpy as np
import os
from datetime import datetime
from tempfile import mkdtemp
from IPython.core.pylabtools import figsize
from math import pi, sin, cos, atan2, sqrt, acos, asin
import pandas as pd
import sympy as sp
from sympy.vector import CoordSys3D
from sympy.geometry import Ray, Circle, intersection
import sympy.matrices
import math
sp.init_printing()

In [ ]:
figsize(16, 10)
mpl.rcParams['lines.markersize'] = 3

# Helper functions and classes

In [ ]:
class Vec2:
    def __init__(self, l0, l1):
        self.l0 = l0
        self.l1 = l1
    
    @property
    def x(self):
        return self.l0
    
    @property
    def y(self):
        return self.l1
        
    @property
    def norm(self):
        return self.r
        
    @property
    def phi(self):
        return atan2(self.y, self.x)
    
    @property
    def r(self):
        return sqrt(self.x**2 + self.y**2)
    
    @property
    def xy(self):
        return (self.l0, self.l1)
    
    @property
    def ixy(self):
        return (self.l1, self.l0)
    
    def __mul__(self, other):
        return Vec2(self.l0*other, self.l1*other)
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    def __truediv__(self, other):
        return Vec2(self.l0/other, self.l1/other)
    
    def __rtruediv__(self, other):
        return self.__div__(other)
    
    def __getitem__(self, key):
        if key == 0: return self.l0
        else: return self.l1
    
    def __add__(self, other):
        return Vec2(self.l0 + other.l0, self.l1 + other.l1)
        
    def __sub__(self, other):
        return Vec2(self.l0 - other.l0, self.l1 - other.l1)
        
    def __repr__(self):
        return "Vec2(l0=%.3f, l1=%.3f)" % (self.l0, self.l1)
        
    def __iter__(self):
        return iter((self.l0, self.l1))
        
    def to_cartesian(self):
        # are polar, will be cartesian
        return Vec2(self.l0*cos(self.l1), self.l0*sin(self.l1))
    
    def to_polar(self):
        # are cart, will be polar
        return Vec2(self.r, self.phi)
    
    def normalized(self):
        return Vec2(self.l0, self.l1) / self.norm
    
    @property
    def xy(self):
        return (self.l0, self.l1)
    
class Line2:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        self.d = b - a
    
    @property
    def xy(self):
        f = np.linspace(0, 1, 10)
        l0 = (self.d*f + self.a).l0
        l1 = (self.d*f + self.a).l1
        return (l0, l1)
    
def boundlim(obj, margin=0.2):
    xmin, ymin, xmax, ymax = obj.bounds
    return {
        "xlim": (xmin-margin, xmax+margin),
        "ylim": (ymin-margin, ymax+margin)
    }

def latex(s, *args):
    def convert(a):
        if type(a).__module__.startswith("sympy"):
            return sp.latex(a)
        return a
    nargs = tuple(map(convert, args))    
    display(Markdown(s % nargs))

class LineArray:
    def __init__(self, lines):
        if type(lines) == list:
            self.lines = lines
        else:
            self.lines = lines.split("\n")
        
    @classmethod
    def fromstring(cls, s):
        return cls(s.split("\n"))
    
    @classmethod
    def center(cls, s, n):
        assert type(s) == str
        lines = [" "*len(s)]*n
        lines[n//2] = s
        return cls(lines)
    
    
    def prefix(self, s):
        return self.__class__([s]*len(self)) + self
    
    def postfix(self, s):
        return self + self.__class__([s]*len(self))
    
    def __and__(self, other):
        assert type(other) == str
        return self.postfix(other)
    
    def __rand__(self, other):
        assert type(other) == str
        return self.prefix(other)
    
    def _pad(self, a, b):
        if len(a) == len(b):
            return a, b
        #smaller = min(a, b, key=lambda e: len(e))
        #bigger = max(a, b, key=lambda e: len(e))
        if len(a) > len(b):
            smaller = b
            bigger = a
            order = 0
        else:
            smaller = a
            bigger = b
            order = 1
        
        pad = len(bigger) - len(smaller)
        top = pad // 2
        bottom = pad - top
        
        assert top + len(smaller) + bottom == len(bigger)
        
        padstr = " "*len(max(smaller, key=lambda s: len(s)))
        
        smaller_padded = [padstr]*top + smaller + [padstr]*bottom
        
        if order == 0:
            return bigger, smaller_padded
        else:
            return smaller_padded, bigger
    
    def __add__(self, other):
        if type(other) == list:
            #assert len(self.lines) == len(other)
            a, b = self._pad(self.lines, other)
            newlines = [i+j for i,j in zip(a, b)]
            return self.__class__(newlines)
        elif type(other) == str:
            return self + self.__class__.center(other, len(self.lines))
        elif isinstance(other, self.__class__):
            return self + other.lines
        else:
            raise NotImplemented()
          
    def __radd__(self, other):
        if type(other) == list:
            return self.__class__(other) + self
        elif type(other) == str:
            return self.__class__.center(other, len(self.lines)) + self
        elif isinstance(other, self.__class__):
            return other + self
        else:
            raise NotImplemented()
        
    def __len__(self):
        return len(self.lines)
    
    def __str__(self):
        return "\n".join(self.lines)
    
def pretty(ex):
    return sp.printing.pretty(ex, use_unicode=False, wrap_line=False)

def jacobian(fs, vs):
    def gen(i, j):
        return sp.Derivative(fs[i], vs[j])
    m = sp.Matrix(len(fs), len(vs), gen)
    return m


def make_cpp_vector(name, tp, values, maxw = 90):
    string = "std::vector<{type}> {name} = {{\n".format(type=tp, name=name)
    line = ""
    for idx, v in enumerate(values):
        if len(line)+len(v) > maxw:
            string += "\t" + line.strip() + "\n"
            line = ""
        line += v
        if idx < len(values)-1:
            line += ", "
    string += "\t" + line.strip()
    string += "\n};"
    return string
def printcpp(s):
    display(Markdown("```cpp\n%s\n```"%s))
    


# Annulus Jacobians

## Jacobian for transform from STRIP to MODULE

The local coordinates given in the calls is in unrotated **STRIP** system. We have an optional additional phi rotation. The rotated **STRIP** coordinates are then shifted in cartesian coordinates, and then converted back to polar coordinates. We call this the **MODULE** system. For this, the covariance needs to be transformed by using a Jacobian.
The covariance $\hat\sigma$ transforms as 

$$\hat\sigma' = J \hat\sigma J^T$$

Set up symbols $x$, $y$ as local cartesian coordinates. The circle in polar coordinates is then:

In [ ]:
x, y, vx, vy = sp.symbols('x y v_x, v_y')
vx, vy = sp.sqrt(x**2 + y**2), 2*sp.atan( y / (sp.sqrt(x**2 + y**2) + x) )
vec = sp.Matrix([vx, vy])

latex(r"$\vec v = %s$", vec)
#print("// " & ("v = " + LineArray(pretty(sp.Matrix(sp.symbols("r' phi'")))) + " = " + LineArray(pretty(vec))))

The Jacobian $J$ is generically:

In [ ]:
fx = sp.Function("f_x")(x, y)
fy = sp.Function("f_y")(x, y)


Jac_gen = jacobian(sp.symbols("f_x f_y"), sp.symbols("x y"))
latex(r"$J_\text{gen} = %s$", Jac_gen)
#print("// " & "Jgen = " + LineArray(pretty(Jac_gen)))


This gives without any further substitutions:

In [ ]:
J = jacobian([vx, vy], [x, y])
latex(r"$$J = %s$$", J)

However, we want to go from **STRIP** to **MODULE** system, which means we need to describe $x, y$ in *shifted* polar coordinates. 

$$
\left(\begin{matrix}
x \\ 
y 
\end{matrix}\right)
= \left(\begin{matrix}
r \cos(\phi - \Delta\phi) \\
r \sin(\phi - \Delta\phi)
\end{matrix}\right)
$$

where $r$, $\phi$ are the polar coordinates of the point and $\Delta\phi$ is a rotation that might be applied.
$x, y$ are still in the **STRIP** system at this point. We transform into **MODULE** by shifting the origin:

$$
\left(
\begin{matrix}
x_2 \\ y_2
\end{matrix}
\right) = 
\left(
\begin{matrix}
x \\ y
\end{matrix}
\right) + \vec O
=
\left(
\begin{matrix}
r \cos(\phi - \Delta\phi) + O_x \\
r \sin(\phi - \Delta\phi + O_y
\end{matrix}
\right)
$$

This vector needs to be calculated back to polar coordinates.

In [ ]:
def make_comments():
    x, y, r, phi, dPhi, O_x, O_y = sp.symbols("x y r phi dPhi O_x O_y")
    a = sp.Matrix([x, y])
    b = sp.Matrix([r*sp.cos(phi - dPhi) + O_x, r*sp.sin(phi - dPhi) + O_y])
    
    print("// " & (LineArray(pretty(a)) + " = " + LineArray(pretty(b))))
    
#make_comments()

In [ ]:
# set up symbols
r_strip, phi_strip, dphi = sp.symbols("r_{strip} \phi_{strip} \Delta\phi")
r_mod, phi_mod = sp.symbols("r_{mod} \phi_{mod}")
Ox, Oy = sp.symbols("O_x O_y")

# calculate polar coordinates for 
x2, y2 = r_strip * sp.cos(phi_strip - dphi) + Ox, r_strip * sp.sin(phi_strip - dphi) + Oy
ml = sp.Matrix([r_mod, phi_mod])
mr = sp.Matrix([vx.subs({x: x2, y: y2}), vy.subs({x: x2, y: y2})])
latex(r"in **MODULE** polar coordinate system:  $%s = %s$", ml, mr)

ml, mr = [m.subs({r_mod: "rMod", phi_mod: "phiMod", r_strip: "rStrip", phi_strip: "phiStrip", dphi: "dPhi"}) for m in (ml, mr)]

#print("// " & ( LineArray(pretty(ml)) + " = " + LineArray(pretty(mr)) ))

In [ ]:
J = jacobian([r_mod, phi_mod], [r_strip, phi_strip])
latex(r"$J = %s$", J)
#print("// " & "J = " + LineArray(pretty(J.subs({r_mod: "rMod", phi_strip: "phiStrip", phi_mod: "phiMod"}))))


latex(r"Substituting $%s$ and $%s$ results in:", r_mod, phi_mod)

J = jacobian([vx.subs({x: x2, y: y2}), vy.subs({x: x2, y: y2})], [r_strip, phi_strip])
s = ""
for i, row in enumerate(J.tolist()):
    for j, e in enumerate(row):
        s += (r"$J_{%d%d} = %s$"+"\n\n") %(i, j, sp.latex(e))

display(Markdown(s))
J = J.applyfunc(lambda c: c.doit())
J = sp.simplify(J)
#latex(r"""
#Carrying out the derivative gives the final jacobian:
#$$J = %s$$
#""", J)

Simplify a bit to ease calculation:

In [ ]:
A, B, C = sp.symbols("A B C")
A_expr = sp.simplify(sp.expand((Ox + r_strip*sp.cos(phi_strip - dphi))**2 
                               + (Oy + r_strip*sp.sin(phi_strip - dphi))**2))
B_expr = sp.cos(phi_strip - dphi)
C_expr = sp.sin(phi_strip - dphi)
Jsim = sp.simplify(sp.expand(J)).subs({A_expr: A, B_expr: B, C_expr: C})

latex("""$$J = %s$$""", Jsim)
#print("J = " + LineArray(pretty((Jsim.subs({r_strip: "rStrip", phi_strip: "phiStrip", dphi: "dPhi"})))))
latex(r"where: $A = %s \\ B = %s \\ C = %s$", A_expr, B_expr, C_expr)

#print("A = " + LineArray(pretty(A_expr.subs({r_strip: "rStrip", phi_strip: "phiStrip", dphi: "dPhi"}))))
#print("B = " + LineArray(pretty(B_expr.subs({r_strip: "rStrip", phi_strip: "phiStrip", dphi: "dPhi"}))))
#print("C = " + LineArray(pretty(C_expr.subs({r_strip: "rStrip", phi_strip: "phiStrip", dphi: "dPhi"}))))

Generate some c++ code to avoid errors by re-typing:

In [ ]:
s = "```cpp\n"
def extocpp(e):
    return sp.printing.cxxcode(e.subs({"r_{strip}": "r_strip", 
                                       "\Delta\phi": "dphi", 
                                       "\phi_{strip}": "phi_strip"}))
s += "double A = " + extocpp(A_expr) + ";\n"
s += "double B = " + extocpp(B_expr) + ";\n"
s += "double C = " + extocpp(C_expr) + ";\n"
s += "Eigen::Matrix<double, 2, 2> J;\n"
for i, row in enumerate(Jsim.tolist()):
    for j, e in enumerate(row):
        s += "J(%d, %d) = "%(i,j) + extocpp(e)
        s += ";\n"
s += "\n```"

display(Markdown(s))

## Jacobian for rotation

Transform into locally rotated PC system. Do we need to transform the covariance for this?

In [ ]:
r_rot, phi_rot = sp.symbols("r_{rot} \phi_{rot}")

J_rot = jacobian([r_rot, phi_rot], [r_strip, phi_strip])
display(J_rot)
J_rot = jacobian([r_strip, phi_strip + dphi], [r_strip, phi_strip])
J_rot = J_rot.applyfunc(lambda c: c.doit())
J_rot

The Jacobian is unity, so we do **not** need to transform covariance.

# Intersection between line and shifted circle

This is needed to find the edge points in **STRIP** system

In [ ]:
x, y, m, b = sp.symbols("x y m b")
r, O_x, O_y = sp.symbols("r O_x O_y")
line_eq = m*x + b
display(line_eq)
circle_eq = (x-O_x)**2 + (y-O_y)**2 - r**2
display(circle_eq)

c_subs = circle_eq.subs(y, line_eq)
display(c_subs)
sols_x = [sp.simplify(sp.expand(s.subs({b: 0}))) for s in sp.solve(c_subs, x)]
display(sols_x)

#for s in [e.subs({b: 0}) for e in sols_x]: 
#    print(sp.printing.cxxcode(s))
#    print("// " & LineArray(pretty(s)))


In [ ]:
mid = Point(0, -0.5)
radius = 1
circ = mid.buffer(radius)
out = Point(2*cos(pi/8), 2*sin(pi/8))
line = LineString([(0, 0), out])
subs = {O_x: mid.x, O_y: mid.y, b: 0, m: out.y/out.x, r: radius}
display(subs)

ix = line.intersection(circ.exterior)

ax = plt.subplot(111, aspect=1, xlim=(-5, 5), ylim=(-5, 5))
ax.add_patch(PolygonPatch(circ, fc="lightgrey"))

ax.plot(*line.xy, color="red")
for p in (ix if type(ix) is list else [ix]):
    print("Point:", p.x, p.y)
    ax.plot(*p.xy, "o", color="orange", markersize=6)

plt.show()

vals_x = [x.subs(subs) for x in sols_x]
vals_y = [sp.solve(line_eq-y, y)[0].subs(subs).subs(x, vx) for vx in vals_x]

pts = [sp.Matrix([x, y]) for x, y in zip(vals_x, vals_y)]
outm = sp.Matrix([out.x, out.y])


the_point, = [p for p in pts if p.dot(sp.Matrix([1, 0])) > 0]
display(the_point)

assert abs(1-ix.x / the_point[0]) < 1e-2
assert abs(1-ix.y / the_point[1]) < 1e-2


# Production of test data for AnnulusDebugAlg

In [ ]:
def make_annulus(minR, maxR, phiMin, phiMax, phiAvg = 0, origin = (0, 0), plot=False, plot_margin=0.2, name="ab"):
    print("phiMin", phiMin, "phiMax", phiMax)
    ox, oy = origin
    oR = sqrt(ox**2 + oy**2)
    ophi = atan2(oy, ox)
    porigin = Point(oR*cos(ophi - phiAvg), oR*sin(ophi - phiAvg))
    
    cin = porigin.buffer(minR)
    cout = porigin.buffer(maxR) 
    
    p1 = (maxR*2*cos(phiMax - phiAvg), maxR*2*sin(phiMax - phiAvg))
    p2 = (maxR*2*cos(phiMin - phiAvg), maxR*2*sin(phiMin - phiAvg))
    
    O = (0, 0)
    poly = Polygon([O, p1, p2])
    p_out = poly.boundary.intersection(cout.boundary)
    p_in = poly.boundary.intersection(cin.boundary)
    
    ring = cout.difference(cin)
    annulus = poly.intersection(ring)
    
    if plot:
        def sp(lay, bounds, aspect=1):
            minx, miny, maxx, maxy = bounds
            ax = plt.subplot(lay, 
                             xlim=(minx-plot_margin, maxx+plot_margin), 
                             ylim=(miny-plot_margin, maxy+plot_margin), 
                             aspect=aspect)
            return ax
        
        
        ax = sp(131, cout.bounds)
        ax.add_patch(PolygonPatch(cout, fc="green"))
        ax.add_patch(PolygonPatch(cin, fc="blue"))
        ax.plot(*porigin.xy, "o", color="red")
        ax.plot(*O, "o", color="red", markerfacecolor='None')
        
        ax = sp(132, cout.union(poly).bounds)
        ax.add_patch(PolygonPatch(ring, fc="lightgrey"))
        
        ax.plot(*porigin.xy, "o", color="red")
        ax.plot(*O, "o", color="red", markerfacecolor='None')
        ax.plot(*poly.exterior.xy)    
        ax.plot([p.x for p in p_out], [p.y for p in p_out], "o", color="orange")
        ax.plot([p.x for p in p_in], [p.y for p in p_in], "o", color="orange")
        
        for p in list(p_in) + list(p_out):
            r = sqrt(p.x**2 + p.y**2)
            phi = atan2(p.y, p.x)
            #print("Point: x,y,r,phi", p.x, p.y, r, phi)
            print(repr(Vec2(p.x, p.y)))

        ax = sp(133, cin.union(annulus).bounds)
        ax.plot(*cin.boundary.xy, "--", color="grey")
        ax.plot(*cout.boundary.xy, "--", color="grey")
        ax.add_patch(PolygonPatch(annulus))
        ax.plot(*porigin.xy, "o", color="red")
        ax.plot(*O, "o", color="red", markerfacecolor='None')
        ax.plot(*LineString([O, p_in[0]]).xy, "b--")
        ax.plot(*LineString([O, p_in[1]]).xy, "b--")

    cpp = """
Trk::AnnulusBoundsPC %s(%f, // minR
                        %f, // maxR
                        %f * M_PI, // phiMin
                        %f * M_PI, // phiMax
                        {%f, %f}, // origin
                        %f * M_PI /* phiAvg */);
"""
    cpp = cpp.strip() % (name, minR, maxR, phiMin/pi, phiMax/pi, ox, oy, phiAvg/pi)
    
    return annulus, cpp
        
annulus, annuluscpp = make_annulus(1, 2, -pi/8, +pi/8, origin = (0, -0.5), phiAvg=0, plot=True, name="asymT1Ab")
plt.show()
display(Markdown("```cpp\n%s\n```"%annuluscpp))

## Distance calculation

The distance that is calculated depends on the coordinate system. The local position argument of the `minDistance` method is expected to be in **STRIP** PC, but it probably makes the most sense to return cartesian distance anyway. In cartesian distance we can decompose the shape and find the closest point on each side.

In [ ]:
def calc_distances(obj_orig, npoints, margin=0.2, trf=lambda x, y: (x, y), itrf=lambda x, y: (x, y)):
    result = []
    obj = transform(trf, obj_orig)
    gminx, gminy, gmaxx, gmaxy = obj_orig.bounds
    for gx in np.linspace(gminx-margin, gmaxx+margin, npoints):
        for gy in np.linspace(gminy-margin, gmaxy+margin, npoints):
            pointpc = Point(*trf(gx, gy))
            pointxy = Point(gx, gy)

            d = obj.boundary.project(pointpc)
            p = obj.boundary.interpolate(d)
            closest_point_coords = list(p.coords)[0]
            cls = Point(*itrf(*closest_point_coords))
            result.append((pointxy, cls))
    return result

In [ ]:
plt.figure().set_figwidth(16)

# number of test points to produce in each coordinate
npoints = 20

# cartesian
ax = plt.subplot(121, aspect=1, xlabel="x", ylabel="y", title="projection in XY")
points = calc_distances(annulus, npoints)
for p, cls in points:
    within = p.within(annulus)
    ax.plot(*p.xy, "o", color="green" if within else "red")
    ax.plot(*cls.xy, "o", color="blue")
    l = LineString([p, cls])
    ax.plot(*l.xy, linewidth=3, color="orange")
ax.add_patch(PolygonPatch(annulus, fc="lightgrey"))

# polar
ax = plt.subplot(122, aspect=1, xlabel="x", ylabel="y", title="projection in PC")
def polartrf(x, y):
    return tuple(Vec2(x, y).to_polar())
def carttrf(l0, l1):
    return tuple(Vec2(l0, l1).to_cartesian())
points_polar = calc_distances(annulus, npoints, trf=polartrf, itrf=carttrf)
annulus_polar = transform(polartrf, annulus)
for p, cls in points_polar: # points are not in polar, distances are
    within = p.within(annulus)
    ax.plot(*p.xy, "o", color="green" if within else "red")
    ax.plot(*cls.xy, "o", color="blue")
    l = LineString([p, cls])
    ax.plot(*l.xy, linewidth=3, color="orange")
ax.add_patch(PolygonPatch(annulus, fc="lightgrey"))
plt.show()

ax = plt.subplot(111, 
                 aspect=1, 
                 xlabel="r", ylabel="\phi",
                 rlim=(0, 3),
                 title="projection in PC",
                 projection="polar")
ax.set_thetamin(-70)
ax.set_thetamax(70)
annulus_pltpc = transform(lambda x, y: tuple(reversed(polartrf(x, y))), annulus)
ax.add_patch(PolygonPatch(annulus_pltpc, fc="lightgrey"))
for p, cls in points_polar: # points are not in polar, distances are
    ppc = Vec2(*reversed(polartrf(p.x, p.y)))
    clspc = Vec2(*reversed(polartrf(cls.x, cls.y)))
    within = p.within(annulus)
    ax.plot(*ppc.xy, "o", color="green" if within else "red")
    ax.plot(*clspc.xy, "o", color="blue")
    l = Line2(ppc, clspc)
    ax.plot(*l.xy, linewidth=3, color="orange")


plt.show()

In [ ]:
# NOT REALLY NEEDED
def polar_translation(p_pc, shift):
    rnew = sqrt(shift.r**2 + p_pc.l0**2 + 2*shift.r*p_pc.l0*cos(p_pc.l1 - shift.phi))
    phinew = acos((p_pc.l0*cos(p_pc.l1) + shift.r*cos(shift.phi)) / rnew)
    return Vec2(rnew, phinew).to_cartesian()
def debug(annulus, margin=0.3):
    xmin, ymin, xmax, ymax = annulus.bounds
    ax = plt.subplot(111, aspect=1, xlim=(xmin-margin, xmax+margin), ylim=(ymin-margin, ymax+margin))
    
    for p, cls in points[45:50]:
        within = p.within(annulus)
        ax.plot(*p.xy, "o", color="green" if within else "red")
        ax.plot(*cls.xy, "o", color="blue")
        l = LineString([p, cls])
        
        shift = -1*Vec2(0, -0.5)
        
        p_strip = Vec2(p.x, p.y)
        p_mod = p_strip + shift
        print("STRIP:", "XY:", p_strip, "PC:", p_strip.to_polar())
        pol_trans = polar_translation(p_strip.to_polar(), shift)
        #print("SHIFTED:", pol_trans)
        v_cls = Vec2(cls.x, cls.y)
        print("CLOSEST:", "XY:", v_cls, "PC:", v_cls.to_polar())
        
        cart_distance = (p_strip - v_cls).norm
        pc_distance = (p_strip.to_polar() - v_cls.to_polar()).norm
        print("cart_distance:", cart_distance, "pc_dist:", pc_distance)
        
        print("---")
        
        ax.plot(*l.xy, linewidth=3, color="orange")
    ax.add_patch(PolygonPatch(annulus, fc="lightgrey"))
    
    corners = [Vec2(l0=0.642, l1=0.266),
               Vec2(l0=0.996, l1=-0.412),
               Vec2(l0=1.619, l1=0.671),
               Vec2(l0=1.972, l1=-0.817)]
    
    mod_lines = [Line2(Vec2(0, -0.5), c) for c in corners]
    for l in mod_lines[:2]:
        ax.plot(*l.xy, color="grey")
    
#debug(annulus)

### Produce C++ output to create the tests more easily

This writes directly to the source file which is included in the test Alg

In [ ]:
name = "asymT1"

cppstr = ""

cppstr += annuluscpp + "\n"*2

cppstr += make_cpp_vector("%sTestPoints"%name, 
                         "Amg::Vector2D", 
                         ["{%s}"%", ".join(map(str, (p.x, p.y))) for p, _ in points])
cppstr += "\n"*2
cppstr += make_cpp_vector("%sClosestPoints"%name, 
                         "Amg::Vector2D", 
                         ["{%s}"%", ".join(map(str, (p.x, p.y))) for _, p in points])
cppstr += "\n"*2
cppstr += make_cpp_vector("%sActDistances"%name, 
                         "double", 
                         [str(LineString([p, cls]).length) for p, cls in points])
cppstr += "\n"*2
cppstr += make_cpp_vector("%sActInsides"%name, 
                         "bool", 
                         ["true" if p.within(annulus) else "false" for p, _ in points], 
                         80)

#printcpp(cppstr)
destfile = "AnnulusDebug/src/AnnulusBoundsPCTestData.h"
if os.path.exists(destfile):
    print("Writing to", os.path.abspath(destfile))
    with open(destfile, "w") as f:
        f.write(cppstr)

# Coordinate transform in polar coordinates
If we want to do quick inside checks, parametrizing $r$ in **MODULE** as a function of $r$ and $\phi$ in **STRIP** system should save some time, since we don't have to do the full blown coordinate transform.

$$
\left(
\begin{matrix}
x \\ y
\end{matrix}
\right)
+
\left(
\begin{matrix}
S_x \\ S_y
\end{matrix}
\right)
=
\left(
\begin{matrix}
x' \\ y'
\end{matrix}
\right)
$$

\begin{align}
r' = \sqrt{x'^2 + y'^2} &= \sqrt{x^2 + S_x^2 + 2x S_x + y^2 + S_y^2 + 2y S_y} \\
&= \sqrt{ r_S^2 + r^2 + 2 r \cdot r_S (\cos\phi \cos\phi' + \sin\phi\sin\phi') } \\
&= \sqrt{ r_S^2 + r^2 + 2r\cdot r_S \cos(\phi - \phi') }
\end{align}

using

\begin{align}
\cos\phi \cos\phi \mp \sin\phi\sin\phi' = \cos(\phi \pm \phi')
\end{align}

The shifted angle $\phi'$ can be calculated as well in principle, but the sign convention seems to be different than what we're used to, and we don't need it for the check anyway.

Verify that this actually works:

In [ ]:
def pol(p1_c, shift):
    p1_pc = p1_c.to_polar()

    
    shift_pc = shift.to_polar()
    
    p2_c = p1_c + shift
    p2_pc = p2_c.to_polar()

    sqrt_arg = shift.r**2 + p1_c.r**2 + 2*shift.r*p1_c.r*cos(p1_c.phi - shift.phi)
    if abs(sqrt_arg) < 1e-10: sqrt_arg = 0
    try:
        rnew = sqrt(sqrt_arg)
    except:
        print(shift.r**2, p1_c.r**2, 2*shift.r*p1_c.r*cos(p1_c.phi - shift.phi), sqrt_arg)
        raise

    p3_pc = Vec2(rnew, 0)
    
    return p2_c, p3_pc.to_cartesian()
    
for sx in range(-5, 5, 100):
    for sy in range(-5, 5, 100):
        for px in range(-5, 5, 100):
            for py in range(-5, 5, 100):
                ref, act = pol(Vec2(px, py), Vec2(sx, sy))
                assert ref.r - act.r < 1e-6, (ref, act)

# Visual testing

This section loads CSV output produced by the AnnulusTestAlg, and displays it.

In [ ]:
_fb = os.path.expanduser(os.path.expandvars("~/devlinks/$HOSTNAME/annulus_21.9/run"))
rundir = os.path.realpath(os.path.abspath(os.environ["ATHRUNDIR"] if "ATHRUNDIR" in os.environ else _fb))
print(rundir)
def read_csv(_file, *args, **kwargs):   
    file = os.path.join(rundir, _file)
    print(file, "modified:", datetime.fromtimestamp(os.path.getmtime(file)))
    if not "sep" in kwargs: kwargs["sep"] = ";"
    return pd.read_csv(file, *args, **kwargs)

In [ ]:
dftol12 = read_csv("vis_tol12.csv")

In [ ]:
figsize(16, 10)

insideAbs = dftol12.loc[dftol12['insideAbs'] == 1]
insideTolExcl = dftol12.loc[(dftol12['insideAbs'] != 1) & (dftol12["insideTol"] == 1)]
insideLoc1Abs = dftol12.loc[(dftol12["insideLoc1Abs"] == 1)]
insideLoc1Tol = dftol12.loc[(dftol12["insideLoc1Tol"] == 1) & (dftol12["insideLoc1Abs"] == 0)]
insideLoc2Abs = dftol12.loc[(dftol12["insideLoc2Abs"] == 1)]
insideLoc2Tol = dftol12.loc[(dftol12["insideLoc2Tol"] == 1) & (dftol12["insideLoc2Abs"] == 0)]

# loc 12
ax = plt.subplot(231, aspect=1, )
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="green"))
ax.scatter(insideAbs["x"], insideAbs["y"], color="blue")
ax.scatter(insideTolExcl["x"], insideTolExcl["y"], color="red")

# loc 1
ax = plt.subplot(232, aspect=1)
ax.set_title("insideLoc1")
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="green"))
ax.scatter(insideLoc1Abs["x"], insideLoc1Abs["y"], color="blue")
ax.scatter(insideLoc1Tol["x"], insideLoc1Tol["y"], color="red")
ax = plt.subplot(235, aspect=1, **boundlim(annulus))
ax.set_title("insideLoc1")
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="green"))
ax.scatter(insideLoc1Abs["x"], insideLoc1Abs["y"], color="blue")
ax.scatter(insideLoc1Tol["x"], insideLoc1Tol["y"], color="red")

# loc 2
ax = plt.subplot(233, aspect=1)
ax.set_title("insideLoc2")
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="green"))
ax.scatter(insideLoc2Abs["x"], insideLoc2Abs["y"], color="blue")
ax.scatter(insideLoc2Tol["x"], insideLoc2Tol["y"], color="red")
ax = plt.subplot(236, aspect=1, **boundlim(annulus))
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="green"))
ax.set_title("insideLoc2")
ax.scatter(insideLoc2Abs["x"], insideLoc2Abs["y"], color="blue")
ax.scatter(insideLoc2Tol["x"], insideLoc2Tol["y"], color="red")

plt.show()

In [ ]:
dfcov = read_csv("vis_cov.csv")

In [ ]:
colors = {0: "grey", 1: "yellow", 2: "green", 3: "pink", 4: "orange"}
dfcov["color"] = dfcov["side"].apply(lambda r: colors[r])

In [ ]:
insideAbs = dfcov[dfcov["inside"] == 1]
insideCov = dfcov[(dfcov["insideCov"] == 1) & (dfcov["inside"] == 0)]

ax = plt.subplot(111, aspect=1)
ax.scatter(insideAbs["x"], insideAbs["y"], color="blue")
#ax.scatter(insideCov["x"], insideCov["y"], color=insideCov["colors"])
insideCov.plot.scatter(x="x", y="y", c=insideCov["color"], ax=ax)
ax.add_patch(PolygonPatch(annulus, alpha=1, fill=False, linewidth=4, ec="red"))
plt.show()

## Comparison to current implementation

In [ ]:
dfcmp = read_csv("vis_cmp.csv")

In [ ]:
insideRef = dfcmp[dfcmp["insideRef"] == 1]
insidePC = dfcmp[dfcmp["inside"] == 1]

insideCorrect = dfcmp[(dfcmp["insideRef"] == dfcmp["inside"]) & (dfcmp["insideRef"] == 1)]
insideIncorrect = dfcmp[dfcmp["insideRef"] != dfcmp["inside"]]

ax = plt.subplot(121, aspect=1)
ax.scatter(insideRef["x"], insideRef["y"], color="grey")
#ax.scatter(insidePC["x"], insidePC["y"], color="blue")

ys = np.linspace(-1, 11)
k_L = -6.6658
k_R = 2.73951
d_L = -1.31112
d_R = 0.556981
ax.plot((ys-d_L)/k_L, ys)
ax.plot((ys-d_R)/k_R, ys)

O_x = (d_L - d_R) / (k_R - k_L)
O_y = O_x*k_L + d_L

ax.plot(*Point(0, 0).buffer(5).exterior.xy)
ax.plot(*Point(0, 0).buffer(10).exterior.xy)
ax.plot(0, 0, "x")
ax.plot(O_x, O_y, "o")

ax.set_xlim(-5, 5)
ax.set_ylim(-1, 11)

ax = plt.subplot(122, aspect=1)
ax.scatter(insideCorrect["x"], insideCorrect["y"], color="green")
ax.scatter(insideIncorrect["x"], insideIncorrect["y"], color="red")
ax.plot((ys-d_L)/k_L, ys)
ax.plot((ys-d_R)/k_R, ys)
ax.plot(*Point(0, 0).buffer(5).exterior.xy)
ax.plot(*Point(0, 0).buffer(10).exterior.xy)
ax.plot(0, 0, "x")
ax.plot(O_x, O_y, "o")
ax.set_xlim(-2, 4)
ax.set_ylim(4, 11)

plt.show()

## Surface class is queried using isOnSurface

The surface needs to be translated from a PlaneSurface, and the transform has to be shifted, so that the origin of **STRIP** is aligned with where the module on the PlaneSurface actually points to.

In [ ]:
dfcmpsrf = read_csv("vis_cmp_srf.csv")
display(dfcmpsrf.head())
display(dfcmpsrf.tail())

In [ ]:
def do_nr(ax, df, nr):
    onSurfaceRef = df[df["onSurfaceRef"] == 1]
    onSurfaceAct = df[df["onSurfaceAct"] == 1]
    onSurfaceCorrect = df[(df["onSurfaceAct"] == df["onSurfaceRef"]) & df["onSurfaceRef"] == 1]
    onSurfaceIncorrect = df[df["onSurfaceRef"] != df["onSurfaceAct"]]

    f = 1
    onSurfaceRef = onSurfaceRef.iloc[::f, :]
    onSurfaceAct = onSurfaceAct.iloc[::f, :]
    onSurfaceCorrect = onSurfaceCorrect.iloc[::f, :]
    onSurfaceIncorrect = onSurfaceIncorrect.iloc[::f, :]

    #ax = fig.add_subplot(141, aspect=1)
    ax.set_xlabel("$l_x$")
    ax.set_ylabel("$l_y$")
    
    ref_corners = list(map(float, df.iloc[0]["ref_corners"].split(",")))
    act_corners = list(map(float, df.iloc[0]["act_corners"].split(",")))
    phiS = df.iloc[0]["phiS"]
    O_x = df.iloc[0]["Ox"]
    O_y = df.iloc[0]["Oy"]
    
    def minmax(a, amin, amax):
        return min(a, amin), max(a, amax)
    
    xmin, xmax = 1e12, -1e12
    ymin, ymax = 1e12, -1e12, 
    
    xmin, xmax = minmax(O_x, xmin, xmax)
    ymin, ymax = minmax(O_y, ymin, ymax)
    
    ppoints = []
    for j in range(3):
        ppoints.append(sp.Point3D(df.iloc[j]["globx"], df.iloc[j]["globy"], df.iloc[j]["globz"]))
    plane = sp.Plane(*ppoints)
    #print(plane)
    #plane = sp.Plane(sp.Point3D(df.iloc[0]["globx"], df.iloc[]
    
    def draw(corners, color, n=0, rot=0, offset=(0, 0)):
        nonlocal xmin, xmax, ymin, ymax
        points_xyz = list(zip(corners[0::3], corners[1::3], corners[2::3]))
        points_xy = [(x, y) for x, y, z in points_xyz]
        
        points_xy = points_xy[n:] + points_xy[:n]
        
        # rotate the corners into local system so we can compare
        for idx in range(len(points_xy)):
            p_orig = Point(points_xy[idx])
            p_moved = Point(p_orig.x + offset[0], p_orig.y + offset[1])
            p_rotated = affinity.rotate(p_moved, rot, origin=(0, 0))
            points_xy[idx] = (p_rotated.x, p_rotated.y)
            
        
        
        l = LineString(points_xy + points_xy[0:1])
        poly = Polygon(points_xy)
        
        left_line = sp.Line(points_xy[0], points_xy[1])
        right_line = sp.Line(points_xy[2], points_xy[3])
        
        ax.plot(*LineString(points_xy[:2]).xy, "o", c=color)
        ax.plot(*LineString(points_xy[2:]).xy, "o", c=color, markerfacecolor="none")
        
        points_xy = list(map(lambda xy: Vec2(*xy), points_xy))
        
        right_dir = (points_xy[0] - points_xy[1]).normalized()
        left_dir = (points_xy[3] - points_xy[2]).normalized()
        
        ix = intersection(left_line, right_line)[0].n()
        xmin, xmax = minmax(ix.x, xmin, xmax)
        ymin, ymax = minmax(ix.y, ymin, ymax)
        ix = ix.x, ix.y
        ax.plot(*ix, "x", c=color)
        
        dminl, dmaxl = -20, 20
        dminr, dmaxr = dminl, dmaxl
        ax.plot(*LineString([ix, points_xy[0]]).xy, "--", c=color)
        ax.plot(*LineString([ix, points_xy[3]]).xy, "--", c=color)
        
        for p in points_xy[0], points_xy[3]:
            xmin, xmax = minmax(p[0], xmin, xmax)
            ymin, ymax = minmax(p[1], ymin, ymax)
                
        ax.add_patch(PolygonPatch(poly, alpha=.1, fill=True, linewidth=0, fc=color))
        
        O = Vec2(0, 0)
        ax.plot(*O.xy, "o", c=color)
        c_in = Point(O.x, O.y).buffer((points_xy[1] - O).norm)
        c_out = Point(O.x, O.y).buffer((points_xy[0] - O).norm)
        ax.plot(*c_in.exterior.xy, "--", c=color)
        ax.plot(*c_out.exterior.xy, "--", c=color)

    draw(ref_corners, color="g")
    draw(act_corners, color="r", n=2, rot=90-phiS/pi*180, offset=(-O_x, -O_y))
    
    for idx, row in onSurfaceCorrect.iterrows():
        xmin, xmax = minmax(row["locx"], xmin, xmax)
        ymin, ymax = minmax(row["locy"], ymin, ymax)
    
    ax.scatter(onSurfaceCorrect["locx"], onSurfaceCorrect["locy"], color="green")
    ax.scatter(onSurfaceIncorrect["locx"], onSurfaceIncorrect["locy"], color="red")
    
    if onSurfaceIncorrect.size == 0:
        ax.text(0.02, 0.97, "full match!", fontsize=12, transform=ax.transAxes, color="green")
    else:
        ax.text(0.02, 0.97, "mismatch!", fontsize=12, transform=ax.transAxes, color="red")
    
    brect = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)])
    xmin, ymin, xmax, ymax = brect.buffer(0.5).exterior.bounds
    
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    
js = range(0, dfcmpsrf["nr"].max()+1, 1)[:20]
gx = 4
gy = math.ceil((len(js)+1) / gx)
figw = 17
figh = gy/gx * figw
fig = plt.figure(figsize=(figw, figh))
for idx, j in enumerate(js):
    ax = fig.add_subplot(gy, gx, idx+1, aspect=1)
    df = dfcmpsrf[dfcmpsrf["nr"] == j]
    print(idx)
    do_nr(ax, df, j)  

plt.show()